In [1]:
import os

In [3]:
%pwd

'/Users/shrikanthnallapati/Projects/stock_analyzer_bot/research'

In [4]:
os.chdir('../')

In [5]:
%pwd

'/Users/shrikanthnallapati/Projects/stock_analyzer_bot'

In [6]:
import os
import time
from bs4 import BeautifulSoup
import re
import requests
import warnings 

import langchain
from langchain.llms import OpenAI
from langchain.agents import load_tools, AgentType, Tool, initialize_agent



In [15]:
# Fetch stock data from Yahoo Finance 
import yfinance as yf

# Fetch stock data from Yahoo Finance

def get_stock_price(ticker,history=5):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    ticker=ticker #+".NS"
    stock = yf.Ticker(ticker)
    df = stock.history(period="1y")
    df=df[["Close","Volume"]]
    df.index=[str(x).split()[0] for x in list(df.index)]
    df.index.rename("Date",inplace=True)
    df=df[-history:]
    # print(df.columns)
    
    return df.to_string()

print(get_stock_price("AMZN"))

                 Close     Volume
Date                             
2023-09-14  144.720001   64033600
2023-09-15  140.389999  102861700
2023-09-18  139.979996   42823500
2023-09-19  137.630005   61482500
2023-09-20  135.289993   46156500


In [16]:
# Script to scrap top5 googgle news for given company name

def google_query(search_term):
    if "news" not in search_term:
        search_term=search_term+" stock news"
    url=f"https://www.google.com/search?q={search_term}&cr=countryUS"
    url=re.sub(r"\s","+",url)
    return url

def get_recent_stock_news(company_name):
    # time.sleep(4) #To avoid rate limit error
    headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'}

    g_query=google_query(company_name)
    res=requests.get(g_query,headers=headers).text
    soup=BeautifulSoup(res,"html.parser")
    news=[]
    for n in soup.find_all("div","n0jPhd ynAwRc tNxQIb nDgy9d"):
        news.append(n.text)
    for n in soup.find_all("div","IJl0Z"):
        news.append(n.text)


    if len(news)>6:
        news=news[:4]
    else:
        news=news
    news_string=""
    for i,n in enumerate(news):
        news_string+=f"{i}. {n}\n"
    top5_news="Recent News:\n\n"+news_string
    
    return top5_news


print(get_recent_stock_news("Microsoft"))

Recent News:

0. Microsoft Eyes Apple's Spot as World's Largest Stock
1. Microsoft Stock Is Still Cheap Given Its Massive Free Cash Flow
2. Buy Microsoft Stock, Analyst Says. AI Will Generate Higher Profits.
3. Microsoft Is a Great Dividend Stock and Just Boosted Its Payout



In [22]:
# Fetch financial statements from Yahoo Finance

def get_financial_statements(ticker):
    # time.sleep(4) #To avoid rate limit error
    if "." in ticker:
        ticker=ticker.split(".")[0]
    else:
        ticker=ticker
    ticker=ticker # +".NS"    
    company = yf.Ticker(ticker)
    balance_sheet = company.balance_sheet
    if balance_sheet.shape[1]>=3:
        balance_sheet=balance_sheet.iloc[:,:3]    # Remove 4th years data
    balance_sheet=balance_sheet.dropna(how="any")
    balance_sheet = balance_sheet.to_string()
    
    cash_flow = company.cash_flow.to_string()
    # print(balance_sheet)
    # print(cash_flow)
    return balance_sheet
    
print(get_financial_statements("AMZN"))

                                                         2022-12-31      2021-12-31      2020-12-31
Treasury Shares Number                                  515000000.0     460000000.0     480000000.0
Ordinary Shares Number                                10242000000.0   10180000000.0   10060000000.0
Share Issued                                          10757000000.0   10640000000.0   10540000000.0
Total Debt                                           140118000000.0  116395000000.0   84389000000.0
Tangible Book Value                                  125755000000.0  122874000000.0   78387000000.0
Invested Capital                                     213193000000.0  186989000000.0  125220000000.0
Working Capital                                       -8602000000.0   19314000000.0    6348000000.0
Net Tangible Assets                                  125755000000.0  122874000000.0   78387000000.0
Capital Lease Obligations                             72968000000.0   67651000000.0   52573000000.0
